<strong>Dependencies installation</strong>

In [2]:
%pip install pandas
%pip install numpy
%pip install tensorflow
%pip install scikit-learn
%pip install scikit-metrics
%pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21484 sha256=3714fd5cc76cbb9411c2a73b10cf6d98b78b387733f156bc5dd2b82c663b0975
  Stored in directory: /home/azureuser/.cache/pip/wheels/e1/e8/83/ddea6

<strong>Dependencies import</strong>

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras import layers, models # type: ignore


/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Importazione del dataset

dataset_url = "https://raw.githubusercontent.com/timothypesi/Data-Sets-For-Machine-Learning-/main/california_housing_train.csv"
df = pd.read_csv(dataset_url)

In [5]:
# Suddivisione del dataset in training set e test set

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [6]:
# Preparazione dei dati per il modello TensorFlow

train_labels = train_df.pop('median_house_value')
test_labels = test_df.pop('median_house_value')

In [8]:
ds = tf.data.Dataset.from_tensor_slices((dict(train_df), train_labels))
ds = ds.batch(32).repeat(200)

feature_ds = ds.map(lambda x, y: x)
# Normalizzazione dei dati

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(feature_ds, batch_size=32, steps=200) #ERROR


# def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
#   def input_function():  # inner function, this will be returned
#     ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
#     if shuffle:
#       ds = ds.shuffle(1000)  # randomize order of data
#     ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
#     return ds  # return a batch of the dataset
#   return input_function  # return a function object for use

ValueError: in user code:

    File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/keras/src/engine/base_preprocessing_layer.py", line 122, in adapt_step  *
        self._adapt_maybe_build(data)
    File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/keras/src/engine/base_preprocessing_layer.py", line 295, in _adapt_maybe_build  **
        self.build(data_shape)
    File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/keras/src/layers/preprocessing/normalization.py", line 173, in build
        input_shape = tf.TensorShape(input_shape).as_list()

    ValueError: as_list() is not defined on an unknown TensorShape.


In [ ]:
# Costruzione del modello ANN con TensorFlow

from sklearn.model_selection import learning_curve


model = models.Sequential([
    normalizer,
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mse')

# Addestramento del modello
#model.fit(train_df, train_labels, epochs=200, validation_split=0.2) 

model.fit(ds, epochs=200) 

#notes:
# loss converges at epoch 199, after that increases again
# SGD not working


In [ ]:
# Valutazione del modello sul test set

loss = model.evaluate(test_df, test_labels)
print(loss)

In [ ]:
# Predizione dei valori sul test set

test_predictions = model.predict(test_df).flatten()

In [ ]:
# Aggiunta delle predizioni al test set

test_df['predicted_median_house_value'] = test_predictions

In [ ]:
# Salvataggio del modello e dei risultati

model.save("Users/g.scorpaniti/notebook/ex4_housing_python/artifacts/model/model.keras")
test_df.to_csv("Users/g.scorpaniti/notebook/ex4_housing_python/artifacts/test_set.csv", index=False)

In [ ]:
# Calcolo degli indicatori R2 e MSA

r2 = r2_score(test_labels, test_predictions)
msa = mean_squared_error(test_labels, test_predictions)

print(f"R2 Score: {r2:.4f}")
print(f"Mean Squared Error: {msa:.4f}")